In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qec import RotatedSurfaceCode

In [2]:
rsc = RotatedSurfaceCode(
    distance = 3,
    depolarize1_rate = 0.01,
    depolarize2_rate = 0.01
)

rsc.build_memory_circuit(number_of_rounds=3)
rsc.memory_circuit.diagram()

q0: -R-DEPOLARIZE1(0.01)-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.01)-----------------------------------------------------------------------------------------------------------------------------------------------------------------@-DEPOLARIZE2(0.01)---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.01)-----------------------------------------------------------------------------------------------------------------------------------------------------------------@-DEPOLARIZE2(0.01)-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------X-DEPOLARIZE2(0.01)-----------------------------------------------------------------------------------------------------------------------------------------------------------------@-DEPOLARIZE2(0.01)--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------DEPOLARIZE1(0.01)------------M:rec[24]------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:
num_shots = 10**2
prob_errors = np.linspace(0, 0.01, 5)

collected_stats = {}

for distance in [3, 5]:
    
    temp_logical_error_rate = []
    
    for prob_error in prob_errors:
        
        rep = RotatedSurfaceCode(
            distance = distance,
            depolarize1_rate = prob_error,
            depolarize2_rate = prob_error
        )
        rep.build_memory_circuit(number_of_rounds=distance*3)
        num_errors_sampled = rep.compute_logical_errors(num_shots)
        temp_logical_error_rate.append(num_errors_sampled / num_shots)
        
    collected_stats[distance] = temp_logical_error_rate

ValueError: The circuit contains non-deterministic detectors.

To make an SVG picture of the problem, you can use the python API like this:
    your_circuit.diagram('detslice-with-ops-svg', tick=range(0, 5), filter_coords=['D82', 'D83', 'D84', 'D85', 'D86', 'D87', 'D88', 'D89', 'D90', 'D91', ])
or the command line API like this:
    stim diagram --in your_circuit_file.stim --type detslice-with-ops-svg --tick 0:5 --filter_coords D82:D83:D84:D85:D86:D87:D88:D89:D90:D91 > output_image.svg

This was discovered while analyzing a Z-basis reset (R) on:
    qubit 12

The collapse anti-commuted with these detectors/observables:
    D82
    D83
    D84
    D85
    D86
    D87
    D88
    D89
    D90
    D91

The backward-propagating error sensitivity for D82 was:
    Z6
    X12

The backward-propagating error sensitivity for D83 was:
    Z6
    X12

The backward-propagating error sensitivity for D84 was:
    Z6
    X12

The backward-propagating error sensitivity for D85 was:
    Z6
    X12

The 

In [ ]:
fig, ax = plt.subplots(1, 1)

for distance in collected_stats.keys():
    ax.plot(prob_errors, collected_stats[distance], label=f"d={distance}")
    
ax.set_ylim(5e-2, 5e-1)
ax.set_xlim(5e-2, 1e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120)